<a href="https://colab.research.google.com/github/lamphgg/Airbnb_filter/blob/main/pandas_project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading the Dataset


In [ ]:
%%capture
!pip install numpy pandas streamlit gdown pyarrow

We will download the datasets from Google Drive just like we did last week, but this time the datasets are in [Pickle](https://pythonnumericalmethods.berkeley.edu/notebooks/chapter11.03-Pickle-Files.html) and [Parquet](https://arrow.apache.org/docs/python/parquet.html) format.

In [ ]:
import os
import shutil

import gdown
import numpy as np
import pandas as pd

# Download files from Google Drive
# Based on data from: http://insideairbnb.com/get-the-data/
file_id_1 = "1m185vTdh-u7_A2ZElBvUD4SCO6oETll2"
file_id_2 = "1w41V1oWHJrBdaNJJQ4oxVBuml5CO7MQX"
downloaded_file_1 = "listings_project.pkl"
downloaded_file_2 = "calendar_project.parquet"
# Download the files from Google Drive
gdown.download(id=file_id_1, output=downloaded_file_1)
gdown.download(id=file_id_2, output=downloaded_file_2)

In [ ]:
pd.set_option("display.max_columns", None)

pd.set_option("display.float_format", "{:.2f}".format)

## Preprocessing the Dataset


In [ ]:
df_list = pd.read_pickle("listings_project.pkl")
df_cal = pd.read_parquet("calendar_project.parquet")

In [ ]:
df_list.head()

In [ ]:
df_cal.head()

In [ ]:
df_list.describe()

In [ ]:
df_cal.describe()

In [ ]:
df_list.info()

In [ ]:
df_list.discount_per_5_days_booked.head(5)

#### Clean


In [ ]:
df_list["discount_per_5_days_booked"] = df_list["discount_per_5_days_booked"].str.replace("%"," ").astype(float)*0.01
df_list["discount_per_10_days_booked"] = df_list["discount_per_10_days_booked"].str.replace("%"," ").astype(float)*0.01
df_list["discount_per_30_and_more_days_booked"] = df_list["discount_per_30_and_more_days_booked"].str.replace("%"," ").astype(float)*0.01

In [ ]:
df_list.discount_per_5_days_booked.head(5)

In [ ]:
df_list[["host_is_superhost", "instant_bookable", "has_availability"]].head(5)

In [ ]:
df_list["host_is_superhost"] = df_list["host_is_superhost"].replace({'f': False, 't': True}).astype(bool)
df_list["instant_bookable"] = df_list["instant_bookable"].replace({'f': False, 't': True}).astype(bool)
df_list["has_availability"] = df_list["has_availability"].replace({'f': False, 't': True}).astype(bool)

In [ ]:
df_list[["host_is_superhost", "instant_bookable", "has_availability"]].head(5)

In [ ]:
df_list[["price", "price_per_person", "minimum_price", 'service_cost']].head(5)

1. Remove dollar signs and commas
1. Convert to `float`



In [ ]:
df_list["price"] = df_list["price"].str.replace("[$,]","",regex=True).astype(float)
df_list["price_per_person"] = df_list["price_per_person"].str.replace("[$,]","",regex=True).astype(float)
df_list["minimum_price"] = df_list["minimum_price"].str.replace("[$,]","",regex=True).astype(float)
df_list["service_cost"] = df_list["service_cost"].str.replace("[$,]","",regex=True).astype(float)

In [ ]:
df_list[["price", "price_per_person", "minimum_price", 'service_cost']].head(5)

#### Change column names

- `price` into `price_in_dollar`
- `neighbourhood_cleansed` into `neighbourhood`  


In [ ]:
df_list = df_list.rename(columns={'price':'price_in_dollar', 'neighbourhood_cleansed':'neighbourhood'})

In [ ]:
df_list[['price_in_dollar','neighbourhood']].head()

In [ ]:
df_list['neighbourhood'] = df_list['neighbourhood'].astype('category')
df_list['room_type'] = df_list['room_type'].astype('category')

#### Delete irrelevant columns
* `host_listings_count`
* `host_total_listings_count`
* `availability_60`
* `availability_90`
* `availability_365`
* `number_of_reviews`
* `number_of_reviews_ltm`
* `reviews_per_month`

In [ ]:
df_list = df_list.drop(columns={"host_listings_count", "host_total_listings_count", "availability_60", "availability_90", "availability_365", "number_of_reviews", "number_of_reviews_ltm", "reviews_per_month"})

In [ ]:
df_list.info(verbose=True, show_counts=True)

In [ ]:
df_list.price_in_euros

In [ ]:
df_list['price_in_euros'].unique()

In [ ]:
df_list = df_list.drop(columns=['price_in_euros'])

In [ ]:
df_list = df_list.dropna(subset=["host_acceptance_rate", "review_scores_rating"])

In [ ]:
df_list.info()

In [ ]:
df_list["room_type"].unique()

['Private room', 'Entire home/apt', 'Hotel room', 'Shared room']
Categories (4, object): ['Entire home/apt', 'Hotel room', 'Private room', 'Shared room']


- **"Private room"** or **"Shared room"** as `room_type`, then the listing only has one bedroom.
- **"Hotel room"** or **"Entire home/apt"** as `room_type`, then the number of guests the listing accomodates by 2 and round up.
- If any of these numbers are missing, then we can leave it empty.

In [ ]:
def fill_empty_bedrooms(accommodates: int, bedrooms: int, room_type: str) -> int:
    if (room_type == "Private room") or (room_type == "Shared room"):
        return 1
    elif (room_type == "Hotel room") or (room_type == "Entire home/apt"):
        return np.ceil(accommodates / 2)
    else:
        return bedrooms

In [ ]:
%%timeit -r 4 -n 100

temp_df = df_list.copy()  # Deep copy of the df, not a "view"
temp_df["rooms"] = df_list[["accommodates", "bedrooms", "room_type"]].apply(
    lambda x: fill_empty_bedrooms(x["accommodates"], x["bedrooms"], x["room_type"]),
    axis=1,
)

In [ ]:
df_list["bedrooms"] = df_list[["accommodates", "bedrooms", "room_type"]].apply(
    lambda x: fill_empty_bedrooms(x["accommodates"], x["bedrooms"], x["room_type"]),
    axis=1,
)

In [ ]:
df_list[["accommodates", "bedrooms", "room_type"]].head()

In [ ]:
%%timeit -r 4 -n 100

temp_df = df_list.copy()

# Please use as many lines as you think you need to
# implement this function. We required 5 separate
# statements.

temp_df["beds"] = temp_df.bedrooms

priv_shared_mask = (
    (temp_df.room_type == "private room") | (temp_df.room_type == "Shared room")
)
temp_df.loc[priv_shared_mask, "beds"] = 1


hotel_apt_mask = (
    (temp_df.room_type == "Hotel room") | (temp_df.room_type == "Entire home/apt")
)
temp_df.loc[hotel_apt_mask, "beds"] = np.ceil(temp_df.accommodates/2)

In [ ]:
df_list =  df_list.dropna(subset=["bedrooms", "beds"])

In [ ]:
df_list["beds"] = df_list["beds"].astype("int")
df_list["bedrooms"] = df_list["bedrooms"].astype("int")

In [ ]:
#Check the current memory usage of the bedrooms column at current int64
df_list["bedrooms"].memory_usage(index=False, deep=True)

In [ ]:
df_list["bedrooms"] = df_list["bedrooms"].astype("int8")
df_list["bedrooms"].memory_usage(index=False, deep=True)

In [ ]:
df_list.info(verbose=True, show_counts=True)

In [ ]:
df_list.head(3)

In [ ]:
# The Calendar DataFrame!
df_cal.head(3)

---

#### Minimum stay

- Create a conditional index for entries `listing_id` that are above the 3 day threshold
- Calculate the price of booking a listing for 5 days by multiplying the current day multiplied by 5, and assign this to a column called `five_day_dollar_price`

In [ ]:
# First start by making a copy, for debugging purposes
calendar_newdf = df_cal.copy()

include_list = (
    calendar_newdf["minimum_nights"] >= 3
)

In [ ]:
# Get all the listings with a minimum nights of 3+
# Use the include_list
calendar_newdf = calendar_newdf.loc[include_list]

| Related functions |
| ---- |
| [isin()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html): Filter the DataFrame on provided values |
| [eq()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.eq.html#pandas.DataFrame.eq): Filter the DataFrame for all values equal to the provided input |
| [ne()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ne.html#pandas.DataFrame.ne): Filter the DataFrame for all values not equal to the provided input |

In [ ]:
calendar_newdf["five_day_dollar_price"] = calendar_newdf["price_in_dollar"] * 5

Transform our newly created DataFrame into a **pivot table**, where we aggregate our rows using the `listing_id` as the index, and the columns `available` and `five_day_dollar_price` as values.

In [ ]:
calendar_summarizeddf = pd.pivot_table(
    data=calendar_newdf,
    index=["listing_id"],
    values=["available", "five_day_dollar_price"],
    aggfunc=np.mean,  # The default aggregation function used
    # for merging multiple related rows of data.
)

calendar_summarizeddf.head(3)

#### Maximum price and date

Make a **pivot table** that states the **maximum `price_in_dollar`** for every Airbnb listing

In [ ]:
temp_sum_df = pd.pivot_table(
   data=calendar_newdf,
    index=["listing_id"],
    values=["price_in_dollar"],
    aggfunc=np.max
)

temp_sum_df.head(3)

#### Task 17: Merging

In [ ]:
final_df = pd.merge(
    df_list,
    calendar_summarizeddf,
    left_on=["id"],
    right_on=["listing_id"],
    how="inner"
)

final_df.head(3)

In [ ]:
final_df.groupby(by=["room_type"])[
    [
        "review_scores_rating",
        "five_day_dollar_price"
    ]
].median()

### Download the Dataset to Your Local Machine

Let's first export our final DataFrame.

In [ ]:
final_df.to_csv(
    "WK2_Airbnb_Amsterdam_listings_proj_solution.csv",
    index=True,
)

In [ ]:
from google.colab import files

# Download the file locally
files.download('WK2_Airbnb_Amsterdam_listings_proj_solution.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>